<a href="https://colab.research.google.com/github/kelvinfkr/Unified_UE_US_cities/blob/main/Illustrative_unified_dataset_traffic_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Assignment Model Implementation

This Colab file provides an implementation of the traffic assignment model with [AequilibraE](https://github.com/AequilibraE/aequilibrae) described in the paper ["A unified dataset for the city-scale traffic assignment model in 20 U.S. cities"](https://doi.org/10.1038/s41597-024-03149-8) by Xu, X., Zheng, Z., Hu, Z. et al. (Sci Data 11, 325 (2024)). The purpose of this Colab file is to facilitate the understanding and learning of traffic assignment concepts for anyone interested in this topic.

## Dataset
The dataset used in this implementation can be found at:
- [Dataset Address](https://figshare.com/ndownloader/files/45086305)

## Contact Information
For inquiries related to the paper, please contact:
- Zhenjie Zheng: zzj17.zheng@polyu.edu.hk
- Wei Ma: wei.w.ma@polyu.edu.hk

The majority of the code in this Colab file is sourced from:
- [GitHub Repository](https://github.com/xuxiaotong/A_unified_and_validated_traffic_dataset_for_20_U.S._cities)

For questions regarding the GitHub code, please contact:
- Xiaotong Xu: kid-a.xu@connect.polyu.hk

If you have any questions or are interested in this Colab file, feel free to reach out to me:
- Kairui Feng: kelvinfkr2015@gmail.com

## Usage
If you are new to Colab or Python, it is recommended to compile this notebook sequentially, line by line, to better understand the code and its functionality.

## License
This code is provided under a non-commercial license. You are free to use, modify, and distribute the code for non-commercial purposes. However, if you use this code in your research or any other work, please make sure to cite the following paper:

Xu, X., Zheng, Z., Hu, Z. et al. (2024). A unified dataset for the city-scale traffic assignment model in 20 U.S. cities. Sci Data 11, 325. https://doi.org/10.1038/s41597-024-03149-8

# download dataset and unzip

In [ ]:
!wget https://figshare.com/ndownloader/files/45086305
!mv 45086305 downloaded_file.zip
!unzip downloaded_file.zip
#this will creat a series of folders under the root


# Install dependency

In [ ]:
!pip install osmnx
!pip install geopandas
# if you want to use system optimal for aequilibrea, do not compile the following line
!pip install aequilibrae
# if you want to get system optimal driven assignment, you need to download the bpr2,pyx
# and substitue that one in the aequilibrae package, paths folder. https://github.com/kelvinfkr/Unified_UE_US_cities/blob/main/bpr2.pyx
# the package aequlibrae is here: https://github.com/AequilibraE/aequilibrae
# the following code shows how to do it.
# !git clone https://github.com/AequilibraE/aequilibrae.git
# then substitue the file
# !wget -O /content/aequilibrae/aequilibrae/paths/bpr2.pyx https://raw.githubusercontent.com/kelvinfkr/Unified_UE_US_cities/main/bpr2.pyx
# Then install the package
# !pip install -e /content/aequilibrae/

#Read the city data

In [ ]:
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import osmnx as ox
import geopandas as gpd
import numpy as np

folders = [
    "01_San_Francisco",
    "02_Seattle",
    "03_Portland",
    "04_Las_Vegas",
    "05_Chicago",
    "06_New_Orleans",
    "07_Austin",
    "08_Minneapolis",
    "09_Dallas",
    "10_Milwaukee",
    "11_New_York",
    "12_Washington",
    "13_Boston",
    "14_Philadelphia",
    "15_Pittsburgh",
    "16_Miami",
    "17_Atlanta",
    "18_Phoenix",
    "19_Denver",
    "20_Honolulu"
]

# if you have other folders with the similar name structure then use the following line to find all the cities.
#import re
#folders = sorted([folder for folder in os.listdir() if re.match(r'^\d+_', folder)], key=lambda x: int(x.split('_')[0]))

# Creat your own folder to store the results

In [ ]:
# List of folder names


# Iterate over each folder
for folder in folders:
    # Change directory to the current folder
    os.chdir(folder)

    # Specify the source and destination folder names
    source_folder = "03_AequilibraE_results"
    destination_folder = "04_my_AequilibraE_results"

    # Check if the source folder exists
    if os.path.exists(source_folder):
        # Copy the folder and its contents to the destination folder
        shutil.copytree(source_folder, destination_folder)
        print(f"Folder '{source_folder}' copied and renamed to '{destination_folder}' successfully in folder '{folder}'.")
    else:
        print(f"Folder '{source_folder}' does not exist in folder '{folder}'.")

    # Change directory back to the parent folder
    os.chdir("..")

#Set Parameters for each city

In [ ]:
# predefine the traffic parameters for each city

road_1_capacity_precondition = [2200,2200,2200,2200,2000,2200,2200,2200,2200,2200,2200,1800,2200,2000,2200,1800,2200,2200,2000,2200]
alpha_precondition = [0.5,0.6,0.5,0.5,0.5,0.6,0.5,0.15,0.6,0.5,0.25,0.5,0.25,0.5,0.5,0.5,0.2,0.15,0.5,0.5]
beta_precondition  = [1.8,3.0,1.2,1.3,1.2,1.8,1.5,1.80,1.3,1.5,1.50,1.5,2.00,1.2,2.0,1.5,1.5,1.20,1.5,1.5]
road_1_capacity_precondition = [2200,2200,2200,2200,2000,2200,2200,2200,2200,2200,2200,1800,2200,2000,2200,1800,2200,2200,2000,2200]
road_1_speed_precondition = [90,90,90,90,90,90,90,90,90,90,90,60,60,90,90,65,70,90,90,90]       # free flow speed (km/h)
road_2_capacity_precondition = [2000,2000,2000,2000,1900,2000,2000,2000,2000,2000,2000,1500,2000,1800,2000,1500,2000,2000,1800,2000]
road_2_speed_precondition = [60,65,65,60,60,60,65,65,65,65,60,40,45,60,60,50,50,65,60,60]
road_3_capacity_precondition = [1400,1400,1400,1400,1400,1400,1400,1300,1400,1400,1400,600,1300,1200,1200,900,1400,1500,1300,1400]
road_3_speed_precondition = [40,45,45,40,40,40,40,40,45,45,40,25,30,30,30,35,35,45,35,40]
od_multiplier_precondition = [0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6]

### if you have other cities, just add a column later, remember to number your city 21,22,23,24,25....

city_params= {
    folder: {
        'alpha': alpha,
        'beta': beta,
        'road_1_capacity': road_1_capacity,
        'road_1_speed': road_1_speed,
        'road_2_capacity': road_2_capacity,
        'road_2_speed': road_2_speed,
        'road_3_capacity': road_3_capacity,
        'road_3_speed': road_3_speed,
        'od_multiplier': od_multiplier
    }
    for folder, alpha, beta, road_1_capacity, road_1_speed, road_2_capacity, road_2_speed, road_3_capacity, road_3_speed, od_multiplier in zip(
        folders,
        alpha_precondition,
        beta_precondition,
        road_1_capacity_precondition,
        road_1_speed_precondition,
        road_2_capacity_precondition,
        road_2_speed_precondition,
        road_3_capacity_precondition,
        road_3_speed_precondition,
        od_multiplier_precondition
    )
}

# you could use something like city_params['01_San_Francisco']['alpha'] to access the detaled parameter for each city

#Define a traffic assignment modular with AequilibraE

In [ ]:
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.paths import Graph
from aequilibrae.paths import TrafficAssignment
from aequilibrae.paths.traffic_class import TrafficClass

# the traffic assignment wrap-up, the input city_path should be something like "01_San_Francisco", a number _ a city name

def traffic_assignment(city_path, vdf_type="BPR"):
    AE_path = city_path + '/' + '04_my_AequilibraE_results'

    ######## step 2: define the od matrix file and generate od index #############
    aemfile = AE_path + '/' + 'od_demand.aem'
    od_data = pd.read_csv(city_path + '/' + '01_Input_data' + '/' + city_path[city_path.index("_") + 1:] + '_od.csv')
    zones = int(max(od_data.O_ID.max() - 9999999, od_data.D_ID.max() - 9999999))
    index = np.arange(zones) + 1

    ####### step 3: define the traffic assignment parameters #####
    connector_speed = 5
    road_1_capacity = city_params[city_path]['road_1_capacity'] # 3 lanes in total, capacity for each lane
    road_1_speed = city_params[city_path]['road_1_speed'] # free flow speed (km/h)
    road_2_capacity = city_params[city_path]['road_2_capacity'] # 2 lanes in total, capacity for each lane
    road_2_speed = city_params[city_path]['road_2_speed']
    road_3_capacity = city_params[city_path]['road_3_capacity'] # capacity for one lane
    road_3_speed = city_params[city_path]['road_3_speed']
    alpha = city_params[city_path]['alpha'] # alpha in BPR function
    beta = city_params[city_path]['beta'] # beta in BPR function
    od_multiplier = city_params[city_path]['od_multiplier'] # OD multiplier

    ######## step 4: select the initial network shapefile path #############
    net = gpd.read_file(AE_path + '/'+ city_path[city_path.index("_") + 1:] + '.shp')
    net.drop('geometry', axis=1, inplace=True)
    net.columns = ["ID", "DIR", "length", "LINK_ID", "a_node", "b_node", "capacity", "LENGTH1", "speed", "Lanes", "link_type"]

    ######## step 5: modify the parameters for traffic assignment #############
    ########### connector ##############
    net.loc[net['link_type'] > 10, 'speed'] = connector_speed
    ########### road class 1 ##############
    net.loc[net['link_type'].isin([1, 2]), 'capacity'] = road_1_capacity
    net.loc[net['link_type'].isin([1, 2]), 'speed'] = road_1_speed
    ########### road class 2 ##############
    net.loc[net['link_type'] == 3, 'capacity'] = road_2_capacity
    net.loc[net['link_type'] == 3, 'speed'] = road_2_speed
    ########### road class 3 ##############
    net.loc[net['link_type'].isin([4, 5]), 'capacity'] = road_3_capacity
    net.loc[net['link_type'].isin([4, 5]), 'speed'] = road_3_speed
    ########### BPR function ##############
    net['free_flow_time'] = net['length'] / net['speed'] * 60
    net['a'] = alpha
    net['b'] = beta

    ########### organize the network data fields ##############
    network = net[['a_node', 'b_node', "capacity", 'free_flow_time', "a", "b"]]
    network = network.assign(direction=1)
    network["link_id"] = network.index + 1
    network = network.astype({"a_node": "int64", "b_node": "int64"})

    ########### step 6: conduct the traffic assignment in AE ##############
    g = Graph()
    g.cost = network['free_flow_time'].values
    g.capacity = network['capacity'].values
    g.free_flow_time = network['free_flow_time'].values
    g.network = network
    g.network_ok = True
    g.status = 'OK'
    g.prepare_graph(index)
    g.set_graph("free_flow_time")
    g.cost = np.array(g.cost, copy=True)
    g.set_skimming(["free_flow_time"])
    g.set_blocked_centroid_flows(False)
    g.network["id"] = g.network.link_id

    aem = AequilibraeMatrix()
    aem.load(aemfile)
    aem.computational_view(["matrix"])

    assigclass = TrafficClass("car", g, aem)
    assig = TrafficAssignment()
    assig.set_classes([assigclass])
    assig.set_vdf(vdf_type)
    assig.set_vdf_parameters({"alpha": "a", "beta": "b"})
    assig.set_capacity_field("capacity")
    assig.set_time_field("free_flow_time")
    assig.set_algorithm("bfw")
    assig.max_iter = 500
    assig.rgap_target = 0.001
    assig.execute()

    assignment_results = assig.results()
    network_data = network.copy()

    ########### step 7: save the output ##############
    network.to_csv(AE_path + '\\' + 'network.csv')
    assig.results().to_csv(AE_path+ '\\' + 'assignment_result.csv')
    return assignment_results, network_data,aem

# Run the traffic assignment

In [ ]:
city_ind = 0 # calculate the UE for the city you like
# if you want to add other cities, name the city as 21_cityname for example and follow the same folder structures
city_path =  folders[city_ind]
assignment_results,network_data,aem = traffic_assignment(city_path)# if you are just aiming for UE, then use this
# if you installed SO and want to use SO
#traffic_assignment(city_path,"BPR2")

# A simple application: average traffic time

In [ ]:
# a simple use of the assignment results, which is to calculate the commuting cost for each vehicle in the system
def calculate_average_traffic_time(city_path, assignment_results, network_data, aem):
    # Step 1: Reset the index of assignment_results and rename the index column to "link_id"
    assignment_results = assignment_results.copy()  # Create a copy to avoid modifying the original DataFrame
    assignment_results.reset_index(inplace=True)
    assignment_results.rename(columns={"index": "link_id"}, inplace=True)

    # Step 2: Merge the assignment results with the network data
    network_data = network_data.merge(assignment_results[["link_id", "VOC_AB", "PCE_tot"]], on="link_id")

    # Step 3: Define the BPR function parameters
    alpha = city_params[city_path]['alpha']  # alpha in BPR function
    beta = city_params[city_path]['beta']  # beta in BPR function

    # Step 4: Calculate the congested travel time using the BPR function, considering connector links
    def calculate_congested_time(row):
        if row["a_node"] > 100000 or row["b_node"] > 100000:  # Connector link
            return row["free_flow_time"]
        else:  # Regular link
            return row["free_flow_time"] * (1 + alpha * (row["VOC_AB"]) ** beta)

    network_data["congested_time"] = network_data.apply(calculate_congested_time, axis=1)

    # Step 5: Load the OD matrix
    od_matrix = aem.matrix["matrix"]

    # Step 6: Calculate the total OD flow
    total_od_flow = od_matrix.sum().sum()

    # Step 7: Calculate the total travel time
    total_travel_time = (network_data["congested_time"] * network_data["PCE_tot"]).sum()

    # Step 8: Calculate the total traffic volume
    total_traffic_volume = network_data["PCE_tot"].sum()

    # Step 9: Calculate the average traffic time
    average_traffic_time = total_travel_time / total_od_flow

    return average_traffic_time

In [ ]:
average_time = calculate_average_traffic_time(city_path, assignment_results, network_data, aem)
print("Average Traffic Time (User Equilibrium):", average_time, "minutes")

Average Traffic Time (User Equilibrium): 24.361353481084972 minutes


# Run over all the cities

In [ ]:
# a simple code to go through all the cities and store the results in a dictionary.
results_store = {}

for city_path in folders:
    assignment_results, network_data, aem = traffic_assignment(city_path)
    average_time = calculate_average_traffic_time(city_path, assignment_results, network_data, aem)
    print(f"Average Traffic Time (User Equilibrium) for {city_path}: {average_time} minutes")

    results_store[city_path] = {
        "assignment_results": assignment_results,
        "network_data": network_data,
        "aem": aem,
        "average_time": average_time
    }

In [ ]:
#if you can not wait, you could use a bit of parallel computing, if you are colab pro user and have 8 kernels.
import multiprocessing

def process_city(city_path):
    assignment_results, network_data, aem = traffic_assignment(city_path)
    average_time = calculate_average_traffic_time(city_path, assignment_results, network_data, aem)
    print(f"Average Traffic Time (User Equilibrium) for {city_path}: {average_time} minutes")

    return {
        "assignment_results": assignment_results,
        "network_data": network_data,
        "aem": aem,
        "average_time": average_time
    }

pool = multiprocessing.Pool()
results = pool.map(process_city, folders)
pool.close()
pool.join()

results_store = dict(zip(folders, results))

In [ ]:
#if you want an so also stored, make sure you changed the bpr2.pyx
results_store = {}

for city_path in folders:
    assignment_results, network_data, aem = traffic_assignment(city_path)
    average_time = calculate_average_traffic_time(city_path, assignment_results, network_data, aem)
    assignment_results_so, network_data_so, aem = traffic_assignment(city_path, "BPR2")
    average_time_so = calculate_average_traffic_time(city_path, assignment_results, network_data, aem)
    print(f"Average Traffic Time (User Equilibrium) for {city_path}: {average_time} minutes")
    print(f"Average Traffic Time (SO) for {city_path}: {average_time_so} minutes")

    results_store[city_path] = {
        "assignment_results": assignment_results,
        "network_data": network_data,
        "aem": aem,
        "average_time": average_time,
        "assignment_results_so": assignment_results_so,
        "network_data_so": network_data_so,
        "average_time_so": average_time_so
    }

In [ ]:
# a parallel version
import multiprocessing

def process_city(city_path):
    assignment_results, network_data, aem = traffic_assignment(city_path)
    average_time = calculate_average_traffic_time(city_path, assignment_results, network_data, aem)

    assignment_results_so, network_data_so, _ = traffic_assignment(city_path, "BPR2")
    average_time_so = calculate_average_traffic_time(city_path, assignment_results_so, network_data_so, aem)

    print(f"Average Traffic Time (User Equilibrium) for {city_path}: {average_time} minutes")
    print(f"Average Traffic Time (SO) for {city_path}: {average_time_so} minutes")

    return {
        "assignment_results": assignment_results,
        "network_data": network_data,
        "aem": aem,
        "average_time": average_time,
        "assignment_results_so": assignment_results_so,
        "network_data_so": network_data_so,
        "average_time_so": average_time_so
    }

pool = multiprocessing.Pool()
results = pool.map(process_city, folders)
pool.close()
pool.join()

results_store = dict(zip(folders, results))